In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timezone 
import json
import matplotlib.pyplot as plt

In [ ]:
# Check your list of packages you've installed in conda.
!conda list

In [ ]:
# load the dataset from Kaggle
df = pd.read_csv('dataset_71k.csv')

In [ ]:
# visualize dataset properties
print("Dataset properties before processing:")
print("Entries:", len(df))
print("Columns:", len(df.columns))
print("Shape:", df.shape)
print("Include columns:", list(df))

In [ ]:
data_df = pd.read_json('txn_data.json')

In [ ]:
data_df.to_csv("blockscout_txn_output.csv", index=False)

In [ ]:
# setting up the dataset
# run here if txn and addr data are present
txn_df = pd.read_csv('blockscout_txn_output.csv')
merged_df = df.merge(txn_df, on='hash')
merged_df.to_csv("merged_data.csv", index=False)

In [ ]:
merged_df['from_address']

In [ ]:
data_df = pd.read_json('addr_data.json')
data_df.to_csv("blockscout_addr_output.csv", index=False)

In [ ]:
addr_df = pd.read_csv('blockscout_addr_output.csv')
addr_df['hash'] = addr_df['hash'].str.lower()

In [ ]:
merged_df_1 = merged_df.merge(addr_df, how='left', left_on='from_address', right_on='hash')
merged_df_2 = merged_df_1.merge(addr_df, how='left', left_on='to_address', right_on='hash')

In [ ]:
merged_df_2.to_csv("merged_dataset.csv", index=False)

In [ ]:
df = None
data_df = None
txn_df = None
merged_df = None
merged_df_1 = None
merged_df_2 = None


df = pd.read_csv("merged_dataset.csv")

In [ ]:
def expand_json_column(df_arg, column_name):
    df_arg[column_name] = df_arg[column_name].str.replace("'", '"')
    # Parse the JSON column into a dictionary
    df_arg[column_name] = df_arg[column_name].apply(json.loads)

    # Get unique keys from the expanded JSON data
    unique_keys = set()
    for row in df_arg[column_name]:
        unique_keys.update(row.keys())

    # Create new columns for each unique key
    for key in unique_keys:
        df_arg[column_name+'_'+key] = df_arg[column_name].apply(lambda x: x.get(key))

    return df_arg

df = expand_json_column(df, 'fee')

In [ ]:
def extract_values(row):
    values = eval(row)  # Evaluate the string as a Python expression
    return values[0], values[1]

df[['confirmation_duration_0', 'confirmation_duration_1']] = df['confirmation_duration'].apply(lambda row: pd.Series(extract_values(row)))

In [ ]:
# change existing column

def calculate_elapsed_time(row):
    elapsed_time = pd.Timestamp.utcnow() - row['timestamp_obj']
    return elapsed_time

df['is_scam'] = df.apply(lambda row: 1 if row['from_scam'] == 1 or row['to_scam'] == 1 else 0, axis=1)
df['timestamp_obj'] = pd.to_datetime(df['block_timestamp'])
df['elapsed_time'] = df.apply(lambda row: calculate_elapsed_time(row), axis=1)
df['tx_token_transfer'] = df.apply(lambda row: 'token_transfer' in row['tx_types'], axis=1)
df['tx_coin_transfer'] = df.apply(lambda row: 'coin_transfer' in row['tx_types'], axis=1)
df['tx_contract_call'] = df.apply(lambda row: 'contract_call' in row['tx_types'], axis=1)

In [ ]:
# change column names and replace duplicated columns to avoid confusion after merging
df.drop(columns=['hash', 'hash_y', 'nonce_y', 'position', 
                'value_y', 
                'gas_limit', 
                'gas_price_y', 
                'raw_input', 
                'gas_used', 
                'block', 
                'timestamp',
                'from_category',
                'to_category',
                'status',
                'method',
                'type',
                'priority_fee',
                'base_fee_per_gas',
                'token_transfers',
                'created_contract',
                'has_error_in_internal_txs',
                'actions',
                'decoded_input',
                'max_priority_fee_per_gas',
                'revert_reason',
                'tx_tag',
                'creation_tx_hash_x',
                'creator_address_hash_x',
                'has_custom_methods_read_x',
                'has_custom_methods_write_x',
                'has_decompiled_code_x',
                'has_logs_x',
                'has_methods_read_x',
                'has_methods_read_proxy_x',
                'has_methods_write_x',
                'has_methods_write_proxy_x',
                'implementation_address_x',
                'implementation_name_x',
                'is_contract_x',
                'is_verified_x',
                'name_x',
                'private_tags_x',
                'public_tags_x',
                'token_x',
                'watchlist_address_id_x',
                'watchlist_names_x',
                'creation_tx_hash_y',
                'creator_address_hash_y',
                'has_custom_methods_read_y',
                'has_custom_methods_write_y',
                'has_decompiled_code_y',
                'has_logs_y',
                'has_methods_read_y',
                'has_methods_read_proxy_y',
                'has_methods_write_y',
                'has_methods_write_proxy_y',
                'implementation_address_y', 
                'implementation_name_y', 
                'is_contract_y', 
                'is_verified_y', 
                'name_y',
                'private_tags_y', 
                'public_tags_y',
                'token_y', 
                'watchlist_address_id_y', 
                'watchlist_names_y',
                'from',
                'to',
                'tx_burnt_fee',
                'max_fee_per_gas',
                'result',
                'fee_type',
                'confirmation_duration',
                'fee',
                 'timestamp_obj',
                 'from_scam',
                 'to_scam',
                 'tx_types',
                 'block_timestamp'
                ], inplace=True)
df.rename(columns={
    'hash_x': 'hash',
    'nonce_x': 'nonce',
    'value_x': 'value',
    'gas_price_x': 'gas_price_txn',
    'block_number_balance_updated_at_x': 'block_number_balance_updated_at_from',
    'block_number_balance_updated_at_y': 'block_number_balance_updated_at_to',
    'coin_balance_x': 'coin_balance_from',
    'coin_balance_y': 'coin_balance_to',
    'exchange_rate': 'exchange_rate_to',
    'exchange_rate_y': 'exchange_rate_from',
    'exchange_rate_x': 'exchange_rate_txn',
    'has_beacon_chain_withdrawals_x': 'has_beacon_chain_withdrawals_from',
    'has_token_transfers_x': 'has_token_transfers_from',
    'has_tokens_x': 'has_tokens_from',
    'has_validated_blocks_x': 'has_validated_blocks_from',
    'has_beacon_chain_withdrawals_y': 'has_beacon_chain_withdrawals_to',
    'has_token_transfers_y': 'has_token_transfers_to',
    'has_tokens_y': 'has_tokens_to',
    'has_validated_blocks_y': 'has_validated_blocks_to',
}, inplace=True)
df.to_csv("20231031_cleaned.csv", index=False)

In [ ]:
df.iloc[0]['elapsed_time']

In [ ]:
df1 = pd.DataFrame({
    "Labels": ["Scam", 'Not Scam'],
    "Is Scam": [df['is_scam'].value_counts()[1], df['is_scam'].value_counts()[0]]
})

scam_count = 0
non_scam_count = 0
for i in range(len(df)):
    if df.iloc[i]['is_scam'] == 1:
        scam_count = scam_count + 1
    else:
        non_scam_count = non_scam_count + 1
df1.plot.pie(y="Is Scam")
print("No. of rows that are not scam:", scam_count)
print("No. of rows that are scams:", non_scam_count)
print("Percentage of scam:", format(scam_count/(scam_count+non_scam_count)*100, ".2f"), "%")
print("Percentage of non scam:", format(non_scam_count/(scam_count+non_scam_count)*100, ".2f"), "%")